# Example notebook: waveform plotting and annotation
This example notebook uses `participants.tsv` and `measurements_oscillometric.tsv` in the `samples` data directory, and the raw waveform data stored in the `measurements_oscillometric` subdirectory.

In [ ]:
# Imports.
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal

In [ ]:
# Location of 'sample' data directory.
DATA_DIR=os.path.join('..', 'sample')

In [ ]:
# Additional constants/parameters.

# Define an offset into the collection of participants.
PID_INDEX = 3

# Define an offset into the collection of measurements.
MEAS_INDEX = 2

# Define the sampling rate for the waveform data (see README.pdf for detail).
FS = 500

In [ ]:
# Load participants.tsv and measurements.tsv dataframes.
ppt_df = pd.read_csv(os.path.join(DATA_DIR, 'participants.tsv'), delimiter='\t')
meas_df = pd.read_csv(os.path.join(DATA_DIR, 'measurements_oscillometric.tsv'), delimiter='\t')

In [ ]:
# Get a view of the participant's measurement metadata.
ppt_meas_df = meas_df.loc[meas_df.pid==ppt_df.pid[PID_INDEX]]

In [ ]:
# Get metadata for a specific measurement.
meas_series = ppt_meas_df.iloc[MEAS_INDEX]
meas_series

In [ ]:
# Load the raw waveform data for the measurement of interest.
waveform_df = pd.read_csv(os.path.join(DATA_DIR, meas_series.waveform_file_path), delimiter='\t')

In [ ]:
# Define ad hoc bandpass filtering function.
def _bandpass_filter(data: np.ndarray, hp: float, lp: float, fs: float = FS) -> np.ndarray:
    order = 4
    b, a = signal.butter(order, (hp, lp), btype='bandpass', fs=fs)
    return signal.lfilter(b, a, data-data[0])

In [ ]:
# Plot ECG, pressure, and optical waveforms. Note that ECG waveform will be inverted or not depending on
# electrode placement.
plt.subplot(3,1,1)
plt.plot(waveform_df.t, _bandpass_filter(waveform_df.ekg.to_numpy(), 0.5, 20))
plt.xlim(0,10)
plt.title(f'{meas_series.measurement} for subject {meas_series.pid} (bp={meas_series.sbp}/{meas_series.dbp})')
plt.ylabel('ECG (mV)')

plt.subplot(3,1,2)
plt.plot(waveform_df.t, _bandpass_filter(waveform_df.pressure.to_numpy(), 0.25, 50))
plt.xlim(0,10)
plt.ylabel('Tonometry (mBAR)')

plt.subplot(3,1,3)
plt.plot(waveform_df.t, _bandpass_filter(waveform_df.optical.to_numpy(), 0.25, 50))
plt.xlim(0,10)
plt.ylabel('PPG (AU)')    